In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics.functional as MF
import dgl
import dgl.nn as dglnn
from dgl.data import AsNodePredDataset
from dgl.dataloading import DataLoader, NeighborSampler, MultiLayerFullNeighborSampler
from ogb.nodeproppred import DglNodePropPredDataset
import tqdm
import argparse
import ast
import sklearn.metrics
import numpy as np
import time
import sys

In [ ]:
dataset = AsNodePredDataset(DglNodePropPredDataset('ogbn-products',root="/home/bear/workspace/singleGNN/data/dataset"))
g = dataset[0]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics.functional as MF
import dgl
import dgl.nn as dglnn
from dgl.data import AsNodePredDataset
from dgl.dataloading import DataLoader, NeighborSampler, MultiLayerFullNeighborSampler
from ogb.nodeproppred import DglNodePropPredDataset
import tqdm
import argparse
import ast
import sklearn.metrics
import numpy as np
import time
import sys
from dgl.dataloading import (
    as_edge_prediction_sampler,
    DataLoader,
    MultiLayerFullNeighborSampler,
    negative_sampler,
    NeighborSampler,
)

In [ ]:
dataset = AsNodePredDataset(DglNodePropPredDataset('ogbn-products',root="/home/bear/workspace/singleGNN/data/dataset"))
g = dataset[0]

In [32]:
import random
# train_src = torch.arange(100).to(torch.int64)
sampled_edge_ids = random.sample(range(g.num_edges()), 10)
neg_sampler = dgl.dataloading.negative_sampler.Uniform(3)
src, dst = neg_sampler(g, torch.Tensor(sampled_edge_ids).to(torch.int64))

In [24]:
sampled_edge_ids = torch.Tensor(sampled_edge_ids).to(torch.int64)
raw_src=g.edges()[0][sampled_edge_ids]
raw_dst=g.edges()[1][sampled_edge_ids]

In [33]:
# print(raw_src==src)
src

tensor([1457698, 1457698, 1457698,  624871,  624871,  624871, 1711490, 1711490,
        1711490,  128035,  128035,  128035,  184691,  184691,  184691,  112917,
         112917,  112917, 1806885, 1806885, 1806885, 1507446, 1507446, 1507446,
         465140,  465140,  465140,  114984,  114984,  114984])

In [29]:
# print(raw_dst==dst)

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False])
